# A Testing Framework: Exercises

## Looping Over `globals`

What happens if you run:

```python
for name in globals():
    print(name)
```

What happens if you run:

```python
name = None
for name in globals():
    print(name)
```

Why?

In [1]:
for name in globals():
    print(name)

__name__


RuntimeError: dictionary changed size during iteration

In [2]:
name = None
for name in globals():
    print(name)

__name__
__doc__
__package__
__loader__
__spec__
__builtin__
__builtins__
_ih
_oh
_dh
In
Out
get_ipython
exit
quit
open
_
__
___
_i
_ii
_iii
_i1
name
_i2


In the first code chunk you are trying to iterate over and modify an object (i.e., dictionary returned by `globals()`) at the same time. The second code chunk doesn't raise a `RuntimeError` because `name` has a global scope to begin with and is not introduced to the global space during the for-loop.

## Counting Results

1.  Modify the test framework so that it reports which tests passed, failed, or had errors
    and also reports a summary of how many tests produced each result.

In [3]:
# copied from lecture slides
def sign(value):
    if value < 0:
        return -1
    else:
        return 1
    
def test_sign_negative():
    assert sign(-3) == -1
def test_sign_positive():
    assert sign(19) == 1
def test_sign_zero():
    assert sign(0) == 0
def test_sign_error():
    assert sgn(1) == 1

In [4]:
# copied from lecture slides
TESTS = [
    test_sign_negative,
    test_sign_positive,
    test_sign_zero,
    test_sign_error
]

In [5]:
# modified function from slide 12
def run_tests(all_tests):
    results = {"pass": [], "fail": [], "error": []}
    for test in all_tests:
        try:
            test()
            results["pass"].append(test.__name__)
        except AssertionError:
            results["fail"].append(test.__name__)
        except Exception:
            results["error"].append(test.__name__)
    
    results['summary'] = {'pass': len(results['pass']),
                          'fail': len(results['fail']),
                          'error': len(results['error'])}
    return(results)

2.  Write unit tests to check that your answer to part 1 works correctly.

In [6]:
result = run_tests(TESTS)

# dictionary has correct length
def test_dict_len():
    assert len(result) == 4

# outer dictionary has expected keys
def test_outer_dict():
    assert result.keys() == {'pass', 'fail', 'error', 'summary'}
    
# inner dictionary has expected keys
def test_outer_dict():
    assert result['summary'].keys() == {'pass', 'fail', 'error'}

In [7]:
test_dict_len()
test_outer_dict()
test_outer_dict()

3.  Think of another plausible way to interpret part 1
    that *wouldn't* pass the tests you wrote for part 2.

Using `print()` instead of `return()`

## Failing on Purpose

Putting assertions into code to check that it is behaving correctly
is called __defensive programming__.
It's a good practice,
but we should make sure those assertions are failing when they're supposed to,
just as we should test our smoke detectors every once in a while.

Modify the tester so that
if a test function's docstring is `"test:assert"`,
the test passes if it raises an `AssertionError`
and fails if it does not.
Tests whose docstring don't contain `"test:assert"`
should behave as before.

In [8]:
def run_tests(all_tests):
    results = {"pass": [], "fail": [], "error": []}
    for test in all_tests:
        if (test.__doc__ == "test:assert"): 
            try:
                test()
            except AssertionError:
                results["pass"].append(test.__name__)
            else:
                results["fail"].append(test.__name__)
        else:
            try:
                test()
                results["pass"].append(test.__name__)
            except AssertionError:
                results["fail"].append(test.__name__)
            except Exception:
                results["error"].append(test.__name__)
    
    results['summary'] = {'pass': len(results['pass']),
                          'fail': len(results['fail']),
                          'error': len(results['error'])}
    return(results)

## Setup and Teardown

Testing frameworks often allow programmers to specify a `setup` function
that is to be run before each test
and a corresponding `teardown` function
that is to be run after each test.
(`setup` usually re-creates complicated test fixtures,
while `teardown` functions are sometimes needed to clean up after tests,
e.g., to close database connections or delete temporary files.)

Modify the testing tool in this chapter so that
if a file of tests contains a function called `setup`
then the tool calls it exactly once before running each test in the file.
Add a similar way to register a `teardown` function.

In [9]:
def run_tests():
    results = {"pass": [], "fail": [], "error": []}
    for (name, test) in globals().items():
        if not name.startswith("test_"):
            continue
            
        if "setup" in globals():
            setup()
        
        if (test.__doc__ == "test:assert"):
            try:
                test()
            except AssertionError:
                results["pass"].append(test.__name__)
            else:
                results["fail"].append(test.__name__)
                
        else:
            try:
                test()
                results["pass"].append(test.__name__)
            except AssertionError:
                results["fail"].append(test.__name__)
            except Exception:
                results["error"].append(test.__name__)
                
        if "teardown" in globals():
            teardown()
        
    results['summary'] = {'pass': len(results['pass']),
                          'fail': len(results['fail']),
                          'error': len(results['error'])}
    return(results)